In [110]:
# Abrir el archivo parquet en un dataframe
import pandas as pd
import numpy as np 

In [111]:
df_final = pd.read_parquet('Dict\df_final.parquet')

In [112]:
df_final.shape

(5100274, 11)

#### Funcion **``1``**

In [113]:
def developer(desarrollador):
    """
    Esta función toma como entrada el nombre de un desarrollador y devuelve un diccionario que contiene información sobre los juegos lanzados por ese desarrollador. 
    La información incluye el número total de juegos lanzados por año, el número de juegos gratuitos lanzados por año y el porcentaje de juegos gratuitos lanzados por año.
    
    Parámetros:
    desarrollador (str): El nombre del desarrollador.
    
    Devuelve:
    dict_developer (dict): Un diccionario que contiene información sobre los juegos lanzados por el desarrollador.

    Ejemplo:
    >>> developer("Nombre del Desarrollador")
    [{'Año': 2010, 'Cantidad de artículos': 5, 'Contenidos Gratis': 2, 'Porcentaje Gratis': '40%'},
     {'Año': 2011, 'Cantidad de artículos': 10, 'Contenidos Gratis': 5, 'Porcentaje Gratis': '50%'},
     {'Año': 'Desconocido', 'Cantidad de artículos': 1, 'Contenidos Gratis': 0, 'Porcentaje Gratis': '0%'}]

    En caso de que el desarrollador no exista, devuelve un diccionario con un mensaje de error.

    Ejemplo:
    >>> developer("Desarrollador Inexistente")
    {
        "detail": [
            {
                "loc": ["string", 0],
                "msg": "El desarrollador no existe",
                "type": "value_error"
            }
        ]
    }
    """
    # Filtrar el dataframe por el desarrollador dado
    df_dev = df_final[df_final['developer'] == desarrollador]

    # Si el dataframe está vacío, lanzar una excepción
    if df_dev.empty:
        return {
            "detail": [
                {
                    "loc": ["string", 0],
                    "msg": "El desarrollador no existe",
                    "type": "value_error"
                }
            ]
        }
    
    # Agrupar por año y contar el número total de juegos únicos y el número de juegos gratuitos únicos
    juegos_por_año = df_dev.groupby('release_date')['item_id'].nunique()
    juegos_gratis_por_año = df_dev[df_dev['price'] == 0.0].groupby('release_date')['item_id'].nunique()
    
    # Asegurarse de que juegos_gratis_por_año tenga la misma longitud que juegos_por_año
    juegos_gratis_por_año = juegos_gratis_por_año.reindex(juegos_por_año.index, fill_value=0)
    
    # Calcular el porcentaje de juegos gratis y redondearlo al entero más cercano
    porcentaje_gratis = ((juegos_gratis_por_año / juegos_por_año) * 100).round().astype(int)
    
    # Asignar nombres a las series
    juegos_por_año.name = "Cantidad de artículos"
    juegos_gratis_por_año.name = "Contenidos Gratis"
    porcentaje_gratis.name = "Porcentaje Gratis"
    
    # Unir las series en un DataFrame
    tabla = pd.concat([juegos_por_año, juegos_gratis_por_año, porcentaje_gratis], axis=1).reset_index()
    tabla.columns = ['Año', 'Cantidad de artículos', 'Contenidos Gratis', 'Porcentaje Gratis']
    
    # Reemplazar 0 con 'Desconocido' en la columna 'Año'
    tabla['Año'] = tabla['Año'].replace(0, 'Desconocido')
    
    # Agregar el signo de porcentaje a 'Porcentaje Gratis'
    tabla['Porcentaje Gratis'] = tabla['Porcentaje Gratis'].apply(lambda x: f"{x}%")
    
    # Convertir el DataFrame a un diccionario
    dict_developer = tabla.to_dict(orient='records')
    
    return dict_developer

In [114]:
# llamar a la función
developer("Adam Foster")

[{'Año': 2013,
  'Cantidad de artículos': 1,
  'Contenidos Gratis': 1,
  'Porcentaje Gratis': '100%'}]

In [115]:
# mostrar developer unicos
df_final['developer'].unique()

array(['Valve', 'Gearbox Software', 'Double Fine Productions', ...,
       'Adam Foster', 'Team Chivalry', 'Empires'], dtype=object)

In [134]:
# buscar desrrolladores con más price igual a 0 y con release_date diferentes y item_id unicos en el dataframe
df_dev = df_final[df_final['price'] == 0.0].groupby('developer').agg({'release_date': 'nunique', 'item_id': 'nunique'}).sort_values(by='release_date', ascending=False)


In [137]:
# user_id unicos en el dataframe con más item_id unicos
df_final.groupby('user_id')['item_id'].nunique().sort_values(ascending=False)

user_id
phrostb              7762
thugnificent         6700
chidvd               6410
piepai               6132
mayshowganmore       5027
                     ... 
76561198108543030       1
76561198108566203       1
76561198108779605       1
76561198108800267       1
76561198100676286       1
Name: item_id, Length: 73753, dtype: int64

#### Funcion **``2``**

In [117]:
def user_data(user_id):
    """
    Esta función devuelve la cantidad de dinero gastado por el usuario, el porcentaje de recomendación
    basado en las revisiones (recommend) y la cantidad de artículos.

    Args:
    - User_id (str): El ID del usuario para el cual se desea obtener la información.

    Returns:
    - dict: Un diccionario que contiene la información del usuario, incluyendo:
        - "Usuario": El ID del usuario.
        - "Dinero gastado": La cantidad de dinero gastado por el usuario en USD.
        - "% de recomendación": El porcentaje de recomendación basado en las revisiones.
        - "cantidad de artículos": La cantidad de artículos únicos que el usuario ha revisado.

    Ejemplo:
    >>> user_data("ID_Usuario")
    {
        'Usuario': 'ID_Usuario',
        'Dinero gastado': '35.0 USD',
        '% de recomendación': '60%',
        'cantidad de artículos': 5
    }

    En caso de que el usuario no exista, devuelve un diccionario con un mensaje de error.

    Ejemplo:
    >>> user_data("Usuario_Inexistente")
    {
        "detail": [
            {
                "loc": ["string", 0],
                "msg": "El usuario Usuario_Inexistente no existe.",
                "type": "value_error"
            }
        ]
    }
    """
    # Filtrar el dataframe 'df_final' por el 'user_id' dado
    df_filtered = df_final[df_final['user_id'] == user_id]

    # Verificar si el dataframe filtrado está vacío
    if df_filtered.empty:
        return {
            "detail": [
                {
                    "loc": ["string", 0],
                    "msg": f"El usuario {user_id} no existe.",
                    "type": "value_error"
                }
            ]
        }

    # Calcular la cantidad de dinero gastado y redondearlo a dos decimales
    total_spent = round(df_filtered['price'].sum(), 2)

    # Calcular el porcentaje de recomendación
    total_reviews = df_filtered['recommend'].count()
    recommended_reviews = df_filtered[df_filtered['recommend'] == True]['recommend'].count()
    recommendation_percentage = round((recommended_reviews / total_reviews) * 100) if total_reviews > 0 else 0

    # Calcular la cantidad de items únicos
    total_items = df_filtered['item_id'].nunique()

    # Crear el diccionario de resultados
    dict_userdata = {
        "Usuario": user_id,
        "Dinero gastado": f"{total_spent} USD",
        "% de recomendación": f"{recommendation_percentage}%",
        "cantidad de artículos": total_items
    }

    return dict_userdata

In [146]:
# llamar a la función
user_data("76561198096849086")

{'Usuario': '76561198096849086',
 'Dinero gastado': '716.42 USD',
 '% de recomendación': '58%',
 'cantidad de artículos': 71}

In [138]:
# mostrar user_id unicos
df_final['user_id'].unique()

array(['76561197970982479', 'js41637', 'evcentric', ...,
       'Fuckfhaisjnsnsjakaka', 'KinkyyyCSGO', 'LydiaMorley'], dtype=object)

#### Funcion **``3``**

In [119]:
def UserForGenre(genero: str):
    # Filtrar el DataFrame por el género dado
    df_genre = df_final[df_final['genres'].apply(lambda x: genero in x)]

    # Si el DataFrame está vacío, lanzar una excepción
    if df_genre.empty:
        return {"detail": [{"loc": ["string", 0], "msg": "El género no existe", "type": "value_error"}]}

    # Obtener el usuario que acumula más horas jugadas para el género dado
    user_most_played_id = df_genre.groupby('user_id')['playtime_forever'].sum().idxmax()

    # Filtrar el DataFrame por el usuario con más horas jugadas para el género dado
    df_user_genre = df_genre[df_genre['user_id'] == user_most_played_id]

    # Obtener la acumulación de horas jugadas por año de lanzamiento
    hours_per_year = get_hours_per_year(df_user_genre)

    # Devolver los resultados en el formato especificado
    return {"Usuario con más horas jugadas para el género {}".format(genero): user_most_played_id, "Horas jugadas": hours_per_year}

def get_hours_per_year(df_genre):
    # Agrupar por año de lanzamiento y sumar las horas jugadas
    hours_per_year = df_genre.groupby('release_date')['playtime_forever'].sum().astype(int).reset_index()
    hours_per_year.columns = ['Año', 'Horas']

    # Cambiar el valor del año a 'desconocido' cuando es igual a 0
    hours_per_year['Año'] = hours_per_year['Año'].apply(lambda x: 'Desconocido' if x == 0 else x)

    # Crear la lista de resultados
    hours_per_year = hours_per_year.to_dict('records')

    return hours_per_year

In [ ]:
# prompt: llamar a la funcion UserForGenre
UserForGenre('Design &amp; Illustration')

In [139]:
# generos unicos
df_final['genres'].unique()

array(["['Action']", "['Strategy']", "['Desconocido']", ...,
       "['Casual', 'Simulation', 'Psychological Horror', 'Horror', 'VR']",
       "['Free to Play', 'MOBA', 'Strategy', 'Multiplayer', 'Team-Based', 'Action', 'e-sports', 'Online Co-Op', 'Competitive', 'PvP', 'RTS', 'Difficult', 'RPG', 'Fantasy', 'Tower Defense', 'Co-op', 'Character Customization', 'Replay Value', 'Action RPG', 'Simulation']",
       "['Action', 'Adventure', 'Massively Multiplayer', 'RPG', 'Simulation', 'Strategy']"],
      dtype=object)

#### Funcion **``4``**

In [121]:
# FUNCIÓN 4 - DESARROLLADOR CON MÁS JUEGOS RECOMENDADOS Y CON COMENTARIOS POSITIVOS
def best_developer_year(year):
    """
    Esta función devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado.
    Se consideran solo los juegos con reviews donde 'recommend' es True y 'sentiment_analysis' es positivo (2).

    Args:
    - año (int): El año para el cual se desea obtener el top de desarrolladores.

    Returns:
    - list: Una lista de diccionarios que contiene el top 3 de desarrolladores con juegos MÁS recomendados por usuarios
      para el año dado. Cada diccionario en la lista tiene el formato {"Puesto X": "Nombre del desarrollador"}.

    Ejemplo:
    >>> best_developer_year(2023)
    [{'Puesto 1': 'Desarrollador A'}, {'Puesto 2': 'Desarrollador B'}, {'Puesto 3': 'Desarrollador C'}]

    En caso de que el año no exista, devuelve un diccionario con un mensaje de error.

    Ejemplo:
    >>> best_developer_year(2020)
    {
        "detail": [
            {
                "loc": ["string", 0],
                "msg": "El año no existe",
                "type": "value_error"
            }
        ]
    }
    """
    # Filtrar el dataframe 'df_final' por release_date, recomendaciones y comentarios positivos
    df_filtered = df_final[(df_final['release_date'] == year) & (df_final['recommend'] == True) & (df_final['sentiment_analysis'] == 2)]

    # cuando el año no existe, lanzar una excepcion
    if year not in df_final['release_date'].unique():
        return {
            "detail": [
                {
                    "loc": ["string", 0],
                    "msg": "El año no existe",
                    "type": "value_error"
                }
            ]
        }

    # Agrupar por desarrollador y contar las reseñas
    developer_counts = df_filtered['developer'].value_counts()

    # Ordenar los resultados y seleccionar los tres primeros
    top_developers = developer_counts.nlargest(3)

    # Crear la lista de resultados
    list_best_developer = []
    for i, (developer, count) in enumerate(top_developers.items(), start=1):
        list_best_developer.append({f"Puesto {i}": developer})

    return list_best_developer

In [122]:
# mostrar release_date unicos
df_final['release_date'].unique()

array([2000, 1999, 2003, 2001, 1998, 2010, 2004, 2005,    0, 2006, 2007,
       2016, 2008, 1997, 2009, 2002, 2011, 2014, 2012, 2013, 2015, 1994,
       1995, 1996, 1993, 1990, 2017, 1991, 1992, 1988, 1983, 1984, 1987,
       1989, 2018], dtype=int64)

In [123]:
# llamar a la función
best_developer_year(2017)

[{'Puesto 1': 'Smartly Dressed Games'},
 {'Puesto 2': 'Freejam'},
 {'Puesto 3': 'Studio Wildcard,Instinct Games,Efecto Studios,Virtual Basement LLC'}]

#### Funcion **``5``**

In [124]:

# FUNCIÓN 5 - ANÁLISIS DE SENTIMIENTO DE RESEÑAS PARA UN DESARROLLADOR
def desarrollador_reviews_analysis(desarrolladora):
    """
    Según el desarrollador, devuelve un diccionario con el nombre del desarrollador como llave
    y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren
    categorizados con un análisis de sentimiento como valor positivo o negativo.

    Args:
    - desarrolladora (str): El nombre del desarrollador para el cual se desea realizar el análisis.

    Returns:
    - dict: Un diccionario que contiene el nombre del desarrollador como llave y una lista con la cantidad
      total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento
      como valor positivo o negativo. Ejemplo de retorno:
      {'Valve': [{'Negativo': 182, 'Positivo': 278}]}

    En caso de que no haya datos para el desarrollador especificado, devuelve un diccionario con un mensaje de error.

    Ejemplo:
    >>> desarrollador_reviews_analysis("Valve")
    {'Valve': [{'Negativo': 182, 'Positivo': 278}]}
    """
    # Filtrar el dataframe por el desarrollador
    df_filtered = df_final[df_final['developer'] == desarrolladora]

    # Verificar si el DataFrame filtrado está vacío y dar una excepción
    if df_filtered.empty:
        return {
            "detail": [
                {
                    "loc": ["string", 0],
                    "msg": "No hay datos para el desarrollador {}".format(desarrolladora),
                    "type": "value_error"
                }
            ]
        }

    # Contar el número de reseñas con análisis de sentimiento positivo y negativo
    sentiment_counts = df_filtered['sentiment_analysis'].value_counts()

    # Crear el diccionario de resultados
    list_reviews_analysis = {desarrolladora: [{'Negativo': int(sentiment_counts.get(0, 0)), 'Positivo': int(sentiment_counts.get(2, 0))}]}

    # Asegurarse de que el resultado es un diccionario
    if not isinstance(list_reviews_analysis, dict):
        list_reviews_analysis = list_reviews_analysis.to_dict()

    return list_reviews_analysis

In [125]:
# llamar a la función
desarrollador_reviews_analysis("Valve")

{'Valve': [{'Negativo': 790, 'Positivo': 206562}]}

In [126]:
# mostrar developer unicos
df_final['developer'].unique()

array(['Valve', 'Gearbox Software', 'Double Fine Productions', ...,
       'Adam Foster', 'Team Chivalry', 'Empires'], dtype=object)

In [127]:
# valores unicos de sentiment_analysis
df_final['sentiment_analysis'].unique()

array([2, 1, 0], dtype=int64)

In [128]:
# mostrar el developer Gearbox Software 
df_final[df_final['developer'] == 'Gearbox Software']

,user_id,item_id,item_name,playtime_forever,recommend,year,sentiment_analysis,genres,release_date,price,developer
4,76561197970982479,50,Half-Life: Opposing Force,0,False,0,1,['Action'],1999,4.99,Gearbox Software
7,76561197970982479,130,Half-Life: Blue Shift,0,False,0,1,['Action'],2001,4.99,Gearbox Software
65,76561197970982479,8980,Borderlands,3061,True,0,2,"['Action', 'RPG']",2009,19.99,Gearbox Software
289,js41637,50,Half-Life: Opposing Force,0,False,0,1,['Action'],1999,4.99,Gearbox Software
291,js41637,130,Half-Life: Blue Shift,0,False,0,1,['Action'],2001,4.99,Gearbox Software
...,...,...,...,...,...,...,...,...,...,...,...
5097166,IRunWithScissors,8980,Borderlands,0,True,2015,2,"['Action', 'RPG']",2009,19.99,Gearbox Software
5097637,Zombie_Assassin,8980,Borderlands,0,True,2015,2,"['Action', 'RPG']",2009,19.99,Gearbox Software
5098152,Dr-Medic,8980,Borderlands,0,True,2014,1,"['Action', 'RPG']",2009,19.99,Gearbox Software
5098870,djhoneybadger,8980,Borderlands,0,True,2013,2,"['Action', 'RPG']",2009,19.99,Gearbox Software


In [129]:
# Contar la cantidad de cada valor en la columna 'review'
df_final['sentiment_analysis'].value_counts()

sentiment_analysis
2    3234465
1    1857231
0       8578
Name: count, dtype: int64

In [130]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5100274 entries, 0 to 5100273
Data columns (total 11 columns):
 #   Column              Dtype  
---  ------              -----  
 0   user_id             object 
 1   item_id             int64  
 2   item_name           object 
 3   playtime_forever    int64  
 4   recommend           bool   
 5   year                int64  
 6   sentiment_analysis  int64  
 7   genres              object 
 8   release_date        int64  
 9   price               float64
 10  developer           object 
dtypes: bool(1), float64(1), int64(5), object(4)
memory usage: 394.0+ MB
